In [1]:
import numpy as np

import pandas as pd

import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping





def prepare_data(path,target_column_start,remove_threshold = 10):

    



    df = pd.read_csv(path, index_col=0)

    

    # delete all feature columns that have less than remove_threshold number of entries

    for col in df.columns:

        if df[col].astype(bool).sum() < remove_threshold:

            df.drop(col,inplace=True,axis=1)

            

    # get all target columns

    target_columns = list(df.columns[list(df.columns).index(target_column_start):])

            

    # get column indices

    col_indices = {col: i for i, col in enumerate(df.columns)}

    target_indices = [col_indices[i] for i in target_columns] 

    

    # split data into train, validation and test

    data_len = len(df)

    train_data = df[0:int(data_len*0.7)]

    val_data = df[int(data_len*0.7):int(data_len*0.9)]

    test_data = df[int(data_len*0.9):]

    num_features = len(target_columns)

    

    return df,target_columns, target_indices,train_data,val_data,test_data, num_features





class create_lookback():

    

    # create lookback class that gives the deep learning model size_input number of elements to predict next eleement from

    

    def __init__(self, size_input,train_data, val_data, test_data,cols=None):



        # initialization of all variables

        self.train_data = train_data

        self.val_data = val_data

        self.test_data = test_data

        self.cols = cols

        

        if cols is not None:

          self.cols_indices = {name: i for i, name in enumerate(cols)}

        self.column_indices = {name: i for i, name in enumerate(train_data.columns)}

        

        self.size_input = size_input

        self.size_lookback = size_input + 1

        self.input_slice = slice(0, size_input)

        self.input_idx = np.arange(self.size_lookback)[self.input_slice]

        self.label_start = self.size_lookback - 1

        self.labels_slice = slice(self.label_start, None)

        self.label_idx = np.arange(self.size_lookback)[self.labels_slice]





    def split(self, features):

        

        # split and reshape dataset  to correct inputs and labels for deep learnin model

        

        inputs = features[:, self.input_slice, :]

        labels = features[:, self.labels_slice, :]

        

        if self.cols is not None:

            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.cols],axis=-1)

    

        inputs.set_shape([None, self.size_input, None])

        labels.set_shape([None, 1, None])



        return inputs, labels







    def create_keras_data(self, data):

        

        # create final data with size_lookback elements, stride movement of 1 and batch_size 16 

        

        data = np.array(data, dtype=np.float32)

        keras_data = tf.keras.preprocessing.timeseries_dataset_from_array( data=data, targets=None, sequence_length=self.size_lookback,

                sequence_stride=1, shuffle=True, batch_size=16,)

        keras_data = keras_data.map(self.split)

        return keras_data



    

    @property

    def train(self):

        return self.create_keras_data(self.train_data)

    

    @property

    def val(self):

        return self.create_keras_data(self.val_data)

    

    @property

    def test(self):

        return self.create_keras_data(self.test_data)







class Baseline(tf.keras.Model):

    #create baseline class that returns back values from previous steps

  def __init__(self, label_idx= [], label_max_count = 9999):

    super().__init__()

    self.label_idx = label_idx

    self.label_max_count = label_max_count



  

  def call(self, inputs):

    if len(self.label_idx) == 0:

      return inputs

    if len(self.label_idx) > 0 and len(self.label_idx)<self.label_max_count:

        result = inputs[:,:, self.label_idx[0]-1:self.label_idx[-1]]

        return result

    result = inputs[:, :, self.label_idx]

    return result[:, :, tf.newaxis]







def model_compile(model, window, patience=10,max_epochs=100):

    # define early stopping

    stop_fcn = EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    # compile deep learning model 

    model.compile(loss=tf.losses.MeanSquaredError(), optimizer=tf.optimizers.Adam(), metrics= [tf.metrics.MeanSquaredError(), tf.metrics.MeanAbsoluteError()])

    # train deep learning model with predefined max epoch, and stopping function 

    out = model.fit(window.train, epochs = max_epochs, validation_data=window.val, callbacks= stop_fcn)

    return model,out





def accuracy_fcn(Y_predicted, Y_groundtruth, threshold = 0.1):



    # threshold  determines which values are considered small enough to be  considered as "not attacked" targets

    # accuracy function for elementwise and setwise accuracy for targer prediction



    cnt = 0

    nonzero_cnt = 0

    cnt_setwise = 0

    nonzero_setwise = 0



    Y_predicted = [[0 if i < threshold else i for i in j] for j in Y_predicted]



    for i in range(len(Y_predicted)):



        Y_pred_curr = Y_predicted[i]

        Y_gt_curr = Y_groundtruth[i]

        

        correct_set = {}

        

        # logic to account for no attack days and to handle target attacked days

                

        if np.count_nonzero(Y_gt_curr) == 0:

            if np.count_nonzero(Y_pred_curr)==0:

                correct_set = [1,1]

                idx_predicted = [None,None]

                idx_groundtruth = [None,None]

            

        elif np.count_nonzero(Y_gt_curr) == 1:

            idx_predicted = np.array(Y_pred_curr).argsort()[-1:][::-1]

            idx_groundtruth = np.array(Y_gt_curr).argsort()[-1:][::-1]

            if np.count_nonzero(Y_pred_curr) == 1:

                idx_predicted = [list(idx_predicted)[0],99]

                idx_groundtruth = [list(idx_groundtruth)[0],99]

            if np.count_nonzero(Y_pred_curr) == 0:

                idx_predicted = [99,99]

                idx_groundtruth = [list(idx_groundtruth)[0],99]

            else:

                idx_predicted = np.array(Y_pred_curr).argsort()[-2:][::-1]

                idx_groundtruth = [list(idx_groundtruth)[0],99]

            correct_set = set(idx_groundtruth).intersection(set(idx_predicted))



        # otherwise just compare top 2 entries 

            

        else:

            idx_predicted = np.array(Y_pred_curr).argsort()[-2:][::-1]

            idx_groundtruth = np.array(Y_gt_curr).argsort()[-2:][::-1]



            correct_set = set(idx_groundtruth).intersection(set(idx_predicted))

            

        if len(correct_set) >= 1:



            cnt_setwise += 1;



        nonzero_setwise += 1;

        cnt += len(correct_set)

        nonzero_cnt += 2



    print('| setwise accuracy = %f' % (cnt / nonzero_cnt))

    print('| eventwise accuracy = %f' % (cnt_setwise / nonzero_setwise))

    return float(cnt / nonzero_cnt),float(cnt_setwise / nonzero_setwise)





def model_eval(model,frame):

    

    # evalation over test data for elementiwse and setwise accuracy 

    predicted_vals = []

    groundtruth_vals = []

    for test_X,test_Y in frame.test:

        out = model(test_X).numpy()

        for i in range(test_Y.shape[0]):

            predicted_vals.append(out[i][0])

            groundtruth_vals.append(test_Y[i][0])

    accuracy_fcn(predicted_vals,groundtruth_vals)

In [2]:
import tensorflow as tf
import random
#from support_functions import model_eval, create_lookback, prepare_data, Baseline
import warnings
import os
warnings.filterwarnings("ignore")
from numpy.random import seed

## Baseline

In [3]:
# random seed set for reproducability
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(5)
seed(5)
random.seed(1254)
tf.random.set_seed(5)

In [4]:
''' import dataset: the script is based on the Afghanistan dataset,
however, one can perform the models using the Iraq one. 
When using the Iraq dataset, target_column_start 
should be as well: 'Private Citizens & Property' '''

path = 'C:\\Users\\Gian Maria\\Desktop\\AAAI_21\\december\\iraq_time_series01.csv' # alternatively: 'iraq_time_series01.csv'
target_column_start =  'Private Citizens & Property'

df,target_columns, target_indices,train_df,val_df,test_df, num_features = prepare_data(path,target_column_start)


# define the lookback 
loockback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)


baseline = Baseline(target_indices,len(df.columns))
model_eval(baseline,loockback_frame)

| setwise accuracy = 0.048780
| eventwise accuracy = 0.094512


## Dense 

### Dense Lookback 1

In [5]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame)
model_eval(dense,lookback_frame)

Epoch 1/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0419 - mean_squared_error: 0.0419 - mean_absolute_error: 0.1017 - val_loss: 0.0303 - val_mean_squared_error: 0.0303 - val_mean_absolute_error: 0.0930
Epoch 2/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0349 - mean_squared_error: 0.0349 - mean_absolute_error: 0.0893 - val_loss: 0.0281 - val_mean_squared_error: 0.0281 - val_mean_absolute_error: 0.0884
Epoch 3/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0343 - mean_squared_error: 0.0343 - mean_absolute_error: 0.0864 - val_loss: 0.0280 - val_mean_squared_error: 0.0280 - val_mean_absolute_error: 0.0879
Epoch 4/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0340 - mean_squared_error: 0.0340 - mean_absolute_error: 0.0848 - val_loss: 0.0284 - val_mean_squared_error: 0.0284 - val_mean_absolute_error: 0.0880
Epoch 5/100
144/144 [==============================] - 1s 4ms/step - loss: 0.0339 - mean_sq

### Dense Lookback 5

In [6]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame5)
model_eval(dense,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0430 - mean_squared_error: 0.0430 - mean_absolute_error: 0.1120 - val_loss: 0.0280 - val_mean_squared_error: 0.0280 - val_mean_absolute_error: 0.0938
Epoch 2/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0341 - mean_squared_error: 0.0341 - mean_absolute_error: 0.0914 - val_loss: 0.0273 - val_mean_squared_error: 0.0273 - val_mean_absolute_error: 0.0872
Epoch 3/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0331 - mean_squared_error: 0.0331 - mean_absolute_error: 0.0862 - val_loss: 0.0264 - val_mean_squared_error: 0.0264 - val_mean_absolute_error: 0.0853
Epoch 4/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0325 - mean_squared_error: 0.0325 - mean_absolute_error: 0.0836 - val_loss: 0.0261 - val_mean_squared_error: 0.0261 - val_mean_absolute_error: 0.0857
Epoch 5/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0320 - mean_squ

### Dense Lookback 15

In [7]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame15)
model_eval(dense,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 1s 9ms/step - loss: 0.0383 - mean_squared_error: 0.0383 - mean_absolute_error: 0.1006 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0924
Epoch 2/100
143/143 [==============================] - 1s 7ms/step - loss: 0.0333 - mean_squared_error: 0.0333 - mean_absolute_error: 0.0871 - val_loss: 0.0262 - val_mean_squared_error: 0.0262 - val_mean_absolute_error: 0.0887
Epoch 3/100
143/143 [==============================] - 1s 6ms/step - loss: 0.0324 - mean_squared_error: 0.0324 - mean_absolute_error: 0.0838 - val_loss: 0.0268 - val_mean_squared_error: 0.0268 - val_mean_absolute_error: 0.0886
Epoch 4/100
143/143 [==============================] - 1s 5ms/step - loss: 0.0316 - mean_squared_error: 0.0316 - mean_absolute_error: 0.0821 - val_loss: 0.0254 - val_mean_squared_error: 0.0254 - val_mean_absolute_error: 0.0881
Epoch 5/100
143/143 [==============================] - 1s 5ms/step - loss: 0.0309 - mean_squ

### Dense Lookback 30

In [8]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model 
dense = tf.keras.Sequential([
    # Shape: (time, features) => (time*features)
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
    # Add back the time dimension.
    # Shape: (outputs) => (1, outputs)
    tf.keras.layers.Reshape([1, -1]),
])

# build and evaluate model
dense, _ = model_compile(dense, lookback_frame30)
model_eval(dense,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 1s 8ms/step - loss: 0.0417 - mean_squared_error: 0.0417 - mean_absolute_error: 0.1098 - val_loss: 0.0278 - val_mean_squared_error: 0.0278 - val_mean_absolute_error: 0.0948
Epoch 2/100
142/142 [==============================] - 1s 4ms/step - loss: 0.0334 - mean_squared_error: 0.0334 - mean_absolute_error: 0.0888 - val_loss: 0.0283 - val_mean_squared_error: 0.0283 - val_mean_absolute_error: 0.0915
Epoch 3/100
142/142 [==============================] - 1s 5ms/step - loss: 0.0320 - mean_squared_error: 0.0320 - mean_absolute_error: 0.0850 - val_loss: 0.0267 - val_mean_squared_error: 0.0267 - val_mean_absolute_error: 0.0914
Epoch 4/100
142/142 [==============================] - 1s 4ms/step - loss: 0.0309 - mean_squared_error: 0.0309 - mean_absolute_error: 0.0831 - val_loss: 0.0266 - val_mean_squared_error: 0.0266 - val_mean_absolute_error: 0.0893
Epoch 5/100
142/142 [==============================] - 1s 7ms/step - loss: 0.0298 - mean_squ

## LSTM

### LSTM Lookback 1

In [9]:
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame)
model_eval(lstm,lookback_frame)

Epoch 1/100
144/144 [==============================] - 2s 17ms/step - loss: 0.0417 - mean_squared_error: 0.0417 - mean_absolute_error: 0.0946 - val_loss: 0.0295 - val_mean_squared_error: 0.0295 - val_mean_absolute_error: 0.0878
Epoch 2/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0361 - mean_squared_error: 0.0361 - mean_absolute_error: 0.0891 - val_loss: 0.0284 - val_mean_squared_error: 0.0284 - val_mean_absolute_error: 0.0868
Epoch 3/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0357 - mean_squared_error: 0.0357 - mean_absolute_error: 0.0877 - val_loss: 0.0284 - val_mean_squared_error: 0.0284 - val_mean_absolute_error: 0.0856
Epoch 4/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0354 - mean_squared_error: 0.0354 - mean_absolute_error: 0.0866 - val_loss: 0.0275 - val_mean_squared_error: 0.0275 - val_mean_absolute_error: 0.0846
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0352 - mean_sq

### LSTM Lookback 5

In [10]:
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame5)
model_eval(lstm,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 2s 17ms/step - loss: 0.0411 - mean_squared_error: 0.0411 - mean_absolute_error: 0.0982 - val_loss: 0.0312 - val_mean_squared_error: 0.0312 - val_mean_absolute_error: 0.0917
Epoch 2/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0360 - mean_squared_error: 0.0360 - mean_absolute_error: 0.0889 - val_loss: 0.0297 - val_mean_squared_error: 0.0297 - val_mean_absolute_error: 0.0905
Epoch 3/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - mean_absolute_error: 0.0870 - val_loss: 0.0286 - val_mean_squared_error: 0.0286 - val_mean_absolute_error: 0.0887
Epoch 4/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0348 - mean_squared_error: 0.0348 - mean_absolute_error: 0.0851 - val_loss: 0.0280 - val_mean_squared_error: 0.0280 - val_mean_absolute_error: 0.0879
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0346 - mean

Epoch 37/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0337 - mean_squared_error: 0.0337 - mean_absolute_error: 0.0816 - val_loss: 0.0253 - val_mean_squared_error: 0.0253 - val_mean_absolute_error: 0.0838
Epoch 38/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0335 - mean_squared_error: 0.0335 - mean_absolute_error: 0.0814 - val_loss: 0.0256 - val_mean_squared_error: 0.0256 - val_mean_absolute_error: 0.0837
Epoch 39/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0336 - mean_squared_error: 0.0336 - mean_absolute_error: 0.0816 - val_loss: 0.0257 - val_mean_squared_error: 0.0257 - val_mean_absolute_error: 0.0836
Epoch 40/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0334 - mean_squared_error: 0.0334 - mean_absolute_error: 0.0813 - val_loss: 0.0259 - val_mean_squared_error: 0.0259 - val_mean_absolute_error: 0.0842
Epoch 41/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0333 - me

#### LSTM Lookback 15

In [11]:
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame15)
model_eval(lstm,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 3s 24ms/step - loss: 0.0393 - mean_squared_error: 0.0393 - mean_absolute_error: 0.1000 - val_loss: 0.0289 - val_mean_squared_error: 0.0289 - val_mean_absolute_error: 0.0926ean_squared_error: 0.0405 - mean_abso
Epoch 2/100
143/143 [==============================] - 2s 13ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - mean_absolute_error: 0.0880 - val_loss: 0.0276 - val_mean_squared_error: 0.0276 - val_mean_absolute_error: 0.0880
Epoch 3/100
143/143 [==============================] - 2s 12ms/step - loss: 0.0346 - mean_squared_error: 0.0346 - mean_absolute_error: 0.0853 - val_loss: 0.0279 - val_mean_squared_error: 0.0279 - val_mean_absolute_error: 0.0875
Epoch 4/100
143/143 [==============================] - 2s 13ms/step - loss: 0.0343 - mean_squared_error: 0.0343 - mean_absolute_error: 0.0841 - val_loss: 0.0275 - val_mean_squared_error: 0.0275 - val_mean_absolute_error: 0.0861
Epoch 5/100
143/143 [==============================

### LSTM Lookback 30

In [12]:
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model 
lstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
lstm, _ = model_compile(lstm, lookback_frame30)
model_eval(lstm,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 8s 59ms/step - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.1005 - val_loss: 0.0293 - val_mean_squared_error: 0.0293 - val_mean_absolute_error: 0.0922
Epoch 2/100
142/142 [==============================] - 4s 31ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - mean_absolute_error: 0.0884 - val_loss: 0.0287 - val_mean_squared_error: 0.0287 - val_mean_absolute_error: 0.0929
Epoch 3/100
142/142 [==============================] - 4s 31ms/step - loss: 0.0347 - mean_squared_error: 0.0347 - mean_absolute_error: 0.0859 - val_loss: 0.0280 - val_mean_squared_error: 0.0280 - val_mean_absolute_error: 0.0887
Epoch 4/100
142/142 [==============================] - 4s 30ms/step - loss: 0.0344 - mean_squared_error: 0.0344 - mean_absolute_error: 0.0843 - val_loss: 0.0276 - val_mean_squared_error: 0.0276 - val_mean_absolute_error: 0.0896
Epoch 5/100
142/142 [==============================] - 5s 32ms/step - loss: 0.0342 - mea

## CNN

### Lookback 1

In [13]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)


conv_width = 1
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame)
model_eval(conv,lookback_frame)

Epoch 1/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0414 - mean_squared_error: 0.0414 - mean_absolute_error: 0.1049 - val_loss: 0.0293 - val_mean_squared_error: 0.0293 - val_mean_absolute_error: 0.0912
Epoch 2/100
144/144 [==============================] - 1s 5ms/step - loss: 0.0353 - mean_squared_error: 0.0353 - mean_absolute_error: 0.0907 - val_loss: 0.0288 - val_mean_squared_error: 0.0288 - val_mean_absolute_error: 0.0880
Epoch 3/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0346 - mean_squared_error: 0.0346 - mean_absolute_error: 0.0873 - val_loss: 0.0279 - val_mean_squared_error: 0.0279 - val_mean_absolute_error: 0.0859
Epoch 4/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0342 - mean_squared_error: 0.0342 - mean_absolute_error: 0.0856 - val_loss: 0.0281 - val_mean_squared_error: 0.0281 - val_mean_absolute_error: 0.0871
Epoch 5/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0339 - mean_sq

### Lookback 5

In [14]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)


conv_width = 5
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame5)
model_eval(conv,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0388 - mean_squared_error: 0.0388 - mean_absolute_error: 0.1020 - val_loss: 0.0269 - val_mean_squared_error: 0.0269 - val_mean_absolute_error: 0.0900
Epoch 2/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0335 - mean_squared_error: 0.0335 - mean_absolute_error: 0.0871 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0890
Epoch 3/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0327 - mean_squared_error: 0.0327 - mean_absolute_error: 0.0836 - val_loss: 0.0261 - val_mean_squared_error: 0.0261 - val_mean_absolute_error: 0.0866
Epoch 4/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0322 - mean_squared_error: 0.0322 - mean_absolute_error: 0.0826 - val_loss: 0.0259 - val_mean_squared_error: 0.0259 - val_mean_absolute_error: 0.0854
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0317 - mean_sq

### LSTM Lookback 15

In [15]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)


conv_width = 15
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame15)
model_eval(conv,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 2s 11ms/step - loss: 0.0381 - mean_squared_error: 0.0381 - mean_absolute_error: 0.0995 - val_loss: 0.0284 - val_mean_squared_error: 0.0284 - val_mean_absolute_error: 0.0909
Epoch 2/100
143/143 [==============================] - 1s 6ms/step - loss: 0.0330 - mean_squared_error: 0.0330 - mean_absolute_error: 0.0864 - val_loss: 0.0281 - val_mean_squared_error: 0.0281 - val_mean_absolute_error: 0.0896
Epoch 3/100
143/143 [==============================] - 1s 8ms/step - loss: 0.0319 - mean_squared_error: 0.0319 - mean_absolute_error: 0.0833 - val_loss: 0.0266 - val_mean_squared_error: 0.0266 - val_mean_absolute_error: 0.0834
Epoch 4/100
143/143 [==============================] - 1s 6ms/step - loss: 0.0311 - mean_squared_error: 0.0311 - mean_absolute_error: 0.0818 - val_loss: 0.0264 - val_mean_squared_error: 0.0264 - val_mean_absolute_error: 0.0820
Epoch 5/100
143/143 [==============================] - 1s 9ms/step - loss: 0.0302 - mean_sq

#### CNN Lookback 30 

In [16]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)


conv_width = 30
conv = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32, kernel_size=(conv_width,), activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv, _ = model_compile(conv, lookback_frame30)
model_eval(conv,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 1s 8ms/step - loss: 0.0378 - mean_squared_error: 0.0378 - mean_absolute_error: 0.1020 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0940
Epoch 2/100
142/142 [==============================] - 1s 4ms/step - loss: 0.0330 - mean_squared_error: 0.0330 - mean_absolute_error: 0.0894 - val_loss: 0.0279 - val_mean_squared_error: 0.0279 - val_mean_absolute_error: 0.0956
Epoch 3/100
142/142 [==============================] - 1s 5ms/step - loss: 0.0316 - mean_squared_error: 0.0316 - mean_absolute_error: 0.0854 - val_loss: 0.0274 - val_mean_squared_error: 0.0274 - val_mean_absolute_error: 0.0912
Epoch 4/100
142/142 [==============================] - 1s 6ms/step - loss: 0.0303 - mean_squared_error: 0.0303 - mean_absolute_error: 0.0832 - val_loss: 0.0268 - val_mean_squared_error: 0.0268 - val_mean_absolute_error: 0.0905
Epoch 5/100
142/142 [==============================] - 1s 4ms/step - loss: 0.0293 - mean_squ

## BILSTM

### Lookback 1

In [17]:
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame)
model_eval(bilstm,lookback_frame)

Epoch 1/100
144/144 [==============================] - 3s 22ms/step - loss: 0.0408 - mean_squared_error: 0.0408 - mean_absolute_error: 0.0946 - val_loss: 0.0284 - val_mean_squared_error: 0.0284 - val_mean_absolute_error: 0.0851
Epoch 2/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0355 - mean_squared_error: 0.0355 - mean_absolute_error: 0.0882 - val_loss: 0.0274 - val_mean_squared_error: 0.0274 - val_mean_absolute_error: 0.0830
Epoch 3/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0353 - mean_squared_error: 0.0353 - mean_absolute_error: 0.0868 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0822
Epoch 4/100
144/144 [==============================] - 1s 6ms/step - loss: 0.0352 - mean_squared_error: 0.0352 - mean_absolute_error: 0.0865 - val_loss: 0.0272 - val_mean_squared_error: 0.0272 - val_mean_absolute_error: 0.0833
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0348 - mean_sq

Epoch 37/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0336 - mean_squared_error: 0.0336 - mean_absolute_error: 0.0816 - val_loss: 0.0255 - val_mean_squared_error: 0.0255 - val_mean_absolute_error: 0.0837
Epoch 38/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0339 - mean_squared_error: 0.0339 - mean_absolute_error: 0.0823 - val_loss: 0.0257 - val_mean_squared_error: 0.0257 - val_mean_absolute_error: 0.0827
Epoch 39/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0339 - mean_squared_error: 0.0339 - mean_absolute_error: 0.0820 - val_loss: 0.0255 - val_mean_squared_error: 0.0255 - val_mean_absolute_error: 0.0830
Epoch 40/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0337 - mean_squared_error: 0.0337 - mean_absolute_error: 0.0818 - val_loss: 0.0256 - val_mean_squared_error: 0.0256 - val_mean_absolute_error: 0.0832
Epoch 41/100
144/144 [==============================] - 1s 10ms/step - loss: 0.0339 -

### BiLSTM Lookback 5

In [18]:
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame5)
model_eval(bilstm,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 4s 30ms/step - loss: 0.0384 - mean_squared_error: 0.0384 - mean_absolute_error: 0.1008 - val_loss: 0.0261 - val_mean_squared_error: 0.0261 - val_mean_absolute_error: 0.0915
Epoch 2/100
144/144 [==============================] - 2s 11ms/step - loss: 0.0344 - mean_squared_error: 0.0344 - mean_absolute_error: 0.0884 - val_loss: 0.0251 - val_mean_squared_error: 0.0251 - val_mean_absolute_error: 0.0873
Epoch 3/100
144/144 [==============================] - 2s 14ms/step - loss: 0.0338 - mean_squared_error: 0.0338 - mean_absolute_error: 0.0852 - val_loss: 0.0247 - val_mean_squared_error: 0.0247 - val_mean_absolute_error: 0.0866
Epoch 4/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0336 - mean_squared_error: 0.0336 - mean_absolute_error: 0.0838 - val_loss: 0.0246 - val_mean_squared_error: 0.0246 - val_mean_absolute_error: 0.0845
Epoch 5/100
144/144 [==============================] - 2s 13ms/step - loss: 0.0335 - mea

### BiLSTM Lookback 15

In [19]:
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame15)
model_eval(bilstm,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 8s 56ms/step - loss: 0.0397 - mean_squared_error: 0.0397 - mean_absolute_error: 0.1075 - val_loss: 0.0265 - val_mean_squared_error: 0.0265 - val_mean_absolute_error: 0.0888
Epoch 2/100
143/143 [==============================] - 3s 21ms/step - loss: 0.0345 - mean_squared_error: 0.0345 - mean_absolute_error: 0.0894 - val_loss: 0.0250 - val_mean_squared_error: 0.0250 - val_mean_absolute_error: 0.0851
Epoch 3/100
143/143 [==============================] - 3s 18ms/step - loss: 0.0339 - mean_squared_error: 0.0339 - mean_absolute_error: 0.0854 - val_loss: 0.0252 - val_mean_squared_error: 0.0252 - val_mean_absolute_error: 0.0848
Epoch 4/100
143/143 [==============================] - 3s 19ms/step - loss: 0.0334 - mean_squared_error: 0.0334 - mean_absolute_error: 0.0835 - val_loss: 0.0255 - val_mean_squared_error: 0.0255 - val_mean_absolute_error: 0.0843
Epoch 5/100
143/143 [==============================] - 3s 19ms/step - loss: 0.0332 - mea

### BiLSTM Lookback 30

In [20]:
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model 
bilstm = tf.keras.models.Sequential([
    # lstm layer
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5)),
    # output layer
    tf.keras.layers.Dense(units=num_features)
])

# build and evaluate model
bilstm, _ = model_compile(bilstm, lookback_frame30)
model_eval(bilstm,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 7s 50ms/step - loss: 0.0398 - mean_squared_error: 0.0398 - mean_absolute_error: 0.1059 - val_loss: 0.0282 - val_mean_squared_error: 0.0282 - val_mean_absolute_error: 0.0922
Epoch 2/100
142/142 [==============================] - 4s 29ms/step - loss: 0.0347 - mean_squared_error: 0.0347 - mean_absolute_error: 0.0890 - val_loss: 0.0269 - val_mean_squared_error: 0.0269 - val_mean_absolute_error: 0.0885
Epoch 3/100
142/142 [==============================] - 5s 34ms/step - loss: 0.0340 - mean_squared_error: 0.0340 - mean_absolute_error: 0.0854 - val_loss: 0.0266 - val_mean_squared_error: 0.0266 - val_mean_absolute_error: 0.0832
Epoch 4/100
142/142 [==============================] - 4s 29ms/step - loss: 0.0338 - mean_squared_error: 0.0338 - mean_absolute_error: 0.0840 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0873
Epoch 5/100
142/142 [==============================] - 4s 30ms/step - loss: 0.0335 - mea

## CLDNN 

### Lookback 1

In [21]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame = create_lookback(1,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 1
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame)
model_eval(conv_lstm,lookback_frame)

Epoch 1/100
144/144 [==============================] - 4s 25ms/step - loss: 0.0410 - mean_squared_error: 0.0410 - mean_absolute_error: 0.0908 - val_loss: 0.0286 - val_mean_squared_error: 0.0286 - val_mean_absolute_error: 0.0878
Epoch 2/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0354 - mean_squared_error: 0.0354 - mean_absolute_error: 0.0876 - val_loss: 0.0282 - val_mean_squared_error: 0.0282 - val_mean_absolute_error: 0.0848
Epoch 3/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0349 - mean_squared_error: 0.0349 - mean_absolute_error: 0.0858 - val_loss: 0.0281 - val_mean_squared_error: 0.0281 - val_mean_absolute_error: 0.0862
Epoch 4/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0346 - mean_squared_error: 0.0346 - mean_absolute_error: 0.0848 - val_loss: 0.0277 - val_mean_squared_error: 0.0277 - val_mean_absolute_error: 0.0857
Epoch 5/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0344 - mean_s

Epoch 37/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0331 - mean_squared_error: 0.0331 - mean_absolute_error: 0.0801 - val_loss: 0.0272 - val_mean_squared_error: 0.0272 - val_mean_absolute_error: 0.0841
Epoch 38/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0331 - mean_squared_error: 0.0331 - mean_absolute_error: 0.0799 - val_loss: 0.0267 - val_mean_squared_error: 0.0267 - val_mean_absolute_error: 0.0842
Epoch 39/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0329 - mean_squared_error: 0.0329 - mean_absolute_error: 0.0796 - val_loss: 0.0265 - val_mean_squared_error: 0.0265 - val_mean_absolute_error: 0.0836
Epoch 40/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0329 - mean_squared_error: 0.0329 - mean_absolute_error: 0.0796 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0832
Epoch 41/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0329 - mea

### CLDNN Lookback 5

In [22]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame5 = create_lookback(5,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 5
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame5)
model_eval(conv_lstm,lookback_frame5)

Epoch 1/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0392 - mean_squared_error: 0.0392 - mean_absolute_error: 0.0918 - val_loss: 0.0274 - val_mean_squared_error: 0.0274 - val_mean_absolute_error: 0.0866
Epoch 2/100
144/144 [==============================] - 1s 8ms/step - loss: 0.0344 - mean_squared_error: 0.0344 - mean_absolute_error: 0.0857 - val_loss: 0.0267 - val_mean_squared_error: 0.0267 - val_mean_absolute_error: 0.0867
Epoch 3/100
144/144 [==============================] - 1s 9ms/step - loss: 0.0337 - mean_squared_error: 0.0337 - mean_absolute_error: 0.0827 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.0872
Epoch 4/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0333 - mean_squared_error: 0.0333 - mean_absolute_error: 0.0817 - val_loss: 0.0265 - val_mean_squared_error: 0.0265 - val_mean_absolute_error: 0.0867
Epoch 5/100
144/144 [==============================] - 1s 7ms/step - loss: 0.0330 - mean_sq

### CLDNN Lookback 15

In [23]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame15 = create_lookback(15,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 15
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame15)
model_eval(conv_lstm,lookback_frame15)

Epoch 1/100
143/143 [==============================] - 4s 30ms/step - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.0931 - val_loss: 0.0280 - val_mean_squared_error: 0.0280 - val_mean_absolute_error: 0.0887
Epoch 2/100
143/143 [==============================] - 1s 9ms/step - loss: 0.0338 - mean_squared_error: 0.0338 - mean_absolute_error: 0.0859 - val_loss: 0.0268 - val_mean_squared_error: 0.0268 - val_mean_absolute_error: 0.0863
Epoch 3/100
143/143 [==============================] - 1s 10ms/step - loss: 0.0331 - mean_squared_error: 0.0331 - mean_absolute_error: 0.0826 - val_loss: 0.0267 - val_mean_squared_error: 0.0267 - val_mean_absolute_error: 0.0858
Epoch 4/100
143/143 [==============================] - 1s 9ms/step - loss: 0.0327 - mean_squared_error: 0.0327 - mean_absolute_error: 0.0813 - val_loss: 0.0269 - val_mean_squared_error: 0.0269 - val_mean_absolute_error: 0.0862
Epoch 5/100
143/143 [==============================] - 1s 8ms/step - loss: 0.0322 - mean_s

### ClDNN Lookback 30

In [24]:
# define the lookback. This is set to 15, but the experiments in the project  have also used 1, 5, and 30
lookback_frame30 = create_lookback(30,train_df,val_df,test_df,target_columns)

# define the deep learning model (conv_width has to be modified in order to be matched with lookback)
conv_width = 30
conv_lstm = tf.keras.Sequential([
    # convolutional layer
    tf.keras.layers.Conv1D(filters=32,kernel_size=(conv_width,),activation='relu'),
    tf.keras.layers.MaxPool1D(pool_size=2,data_format='channels_first'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    # lstm layer
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.5),
    tf.keras.layers.Dense(units=num_features),
])

# build and evaluate model
conv_lstm, _ = model_compile(conv_lstm, lookback_frame30)
model_eval(conv_lstm,lookback_frame30)

Epoch 1/100
142/142 [==============================] - 4s 26ms/step - loss: 0.0389 - mean_squared_error: 0.0389 - mean_absolute_error: 0.0947 - val_loss: 0.0263 - val_mean_squared_error: 0.0263 - val_mean_absolute_error: 0.0846
Epoch 2/100
142/142 [==============================] - 1s 10ms/step - loss: 0.0342 - mean_squared_error: 0.0342 - mean_absolute_error: 0.0858 - val_loss: 0.0260 - val_mean_squared_error: 0.0260 - val_mean_absolute_error: 0.0853
Epoch 3/100
142/142 [==============================] - 1s 6ms/step - loss: 0.0332 - mean_squared_error: 0.0332 - mean_absolute_error: 0.0829 - val_loss: 0.0260 - val_mean_squared_error: 0.0260 - val_mean_absolute_error: 0.0825
Epoch 4/100
142/142 [==============================] - 1s 8ms/step - loss: 0.0325 - mean_squared_error: 0.0325 - mean_absolute_error: 0.0815 - val_loss: 0.0259 - val_mean_squared_error: 0.0259 - val_mean_absolute_error: 0.0822
Epoch 5/100
142/142 [==============================] - 1s 6ms/step - loss: 0.0318 - mean_s